This notebook will serve as the master notebook which the TAs can run from start to finish

In [1]:
import pandas as pd
import numpy as np
from src.utils import get_batches, shuffle, train_val_split


%load_ext autoreload
%autoreload 2

In [2]:
# Define the path to the data
data_path = './data/essay_df.pkl'
df = pd.read_pickle(data_path)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8764 entries, 0 to 10683
Data columns (total 8 columns):
essay_id         8764 non-null int64
essay_set        8764 non-null int64
essay            8764 non-null object
domain1_score    8764 non-null int64
essays_embed     8764 non-null object
word_count       8764 non-null int64
max_score        8764 non-null float64
norm_score       8764 non-null float64
dtypes: float64(2), int64(4), object(2)
memory usage: 616.2+ KB


In [14]:
X = np.array(df['essays_embed'])
y = np.array(df['norm_score'])

In [15]:
X_stacked = np.stack(X, axis=0)
print(X_stacked.shape)

(8764, 600, 100)


In [16]:
X_flat = np.reshape(X_stacked, [X_stacked.shape[0], -1])
print(X_flat.shape)

(8764, 60000)


In [17]:
X_shuffled, y_shuffled = shuffle(X_flat, y)

In [18]:
print(X_shuffled.shape)

(8764, 60000)


In [19]:
X_train, y_train, X_val, y_val = train_val_split(X_shuffled, y_shuffled, train_prop=0.8)

In [51]:
from src.mlp import MLP
input_dim = X_train.shape[1]
batch_size = 100

batch_gen = get_batches(X_train, y_train, batch_size, net_type='mlp')

my_net = MLP(input_dim=input_dim, hidden_dims=[100, 100], num_classes=12, l2_reg=1e-4)

In [52]:
my_net.train(gen=batch_gen, X_val=X_val, y_val=y_val, n_epochs=20, lr=1e-2)

loss for counter 5 is 2.438948392868042
counter 5: valid acc = 0.0
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 10 is 2.4793944358825684
counter 10: valid acc = 0.0
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 15 is 2.471740245819092
counter 15: valid acc = 0.0
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 20 is 1.693276286125183
counter 20: valid acc = 0.0
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 25 is 1.7523139715194702
counter 25: valid acc = 0.0
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


...

loss for counter 1000 is 0.8452533483505249
counter 1000: valid acc = 0.0
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 1005 is 0.7553697824478149
counter 1005: valid acc = 0.0
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 1010 is 0.975233793258667
counter 1010: valid acc = 0.0
[8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8 8]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


loss for counter 1015 is 1.4367343187332153
counter 1015: valid acc = 0.0
[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4]
[12  9  9  6  9  9  6  9  9  3  9  6 12  6 12  9 12  3  9  3]


KeyboardInterrupt: 